In [1]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

In [2]:

rospy.init_node('test_ur', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'gripper']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])

In [4]:
planning_frame = move_group.get_planning_frame()

In [5]:
planning_frame

'world'

In [6]:
eef_link = move_group.get_end_effector_link()

In [7]:
eef_link

'robotiq_85_base_link'

In [8]:

robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, robotiq_85_left_knuckle_joint]
  position: [1.126471318939787, 1.0207072076158932, 1.397606134512059, 0.9402977512464314, -1.0981139419616763, -1.0746002898211957, 0.0006718933906437741]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [9]:

move_group.set_named_target("up")
plan = move_group.plan()

In [11]:
move_group.execute(plan, wait=True)

False

In [12]:

def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_state = move_group.get_current_joint_values()
    print "---- rad value ----"
    print joint_state
    print "---- degree value ---- "
    print [joint*180/math.pi for joint in joint_state]

In [13]:

import math

get_joint_state(move_group)

---- rad value ----
[-0.00021188807999905634, -1.2666327103141297, 7.224502267888511e-05, -9.315989488811738e-05, -8.809645722251958e-05, 9.166754702327751e-05]
---- degree value ---- 
[-0.01214029271307628, -72.57270849421626, 0.004139334890327033, -0.005337668796971499, -0.005047555188905171, 0.005252163562750814]


In [14]:
goal = [ n * math.pi / 180 for n in [90,-100,90,-100,-90,0]]

move_joints(move_group, goal)
get_joint_state(move_group)

---- rad value ----
[1.5707463470828795, -1.7452533336392406, 1.5707591248619268, -1.745229791004168, -1.5707991763504001, -8.068717061249231e-05]
---- degree value ---- 
[89.99713637344014, -99.99565019866583, 89.99786848625111, -99.99430130503755, -90.00016326750384, -0.004623034336947815]


In [15]:
# cartesin value retrieval

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [16]:

get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 452
    nsecs: 401000000
  frame_id: "world"
pose: 
  position: 
    x: -0.109128144121
    y: 0.432177861605
    z: 1.52177845008
  orientation: 
    x: -0.405633114166
    y: 0.405604190356
    z: 0.57920169955
    w: 0.579199800325


In [17]:

import geometry_msgs.msg

pose_goal = geometry_msgs.msg.Pose()

pose_goal.position.x = 0.562
pose_goal.position.y = 0.550
pose_goal.position.z = 1.37679003904

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

아래 있는 cell 이 quaternion orientation 이 제대로 잡혀 있음... 여기에서 contraint 를 한번 걸어보는 것으로 해보기 ... 

In [64]:
pose_goal.position.x = 0.45
pose_goal.position.y = -0.44
pose_goal.position.z = 0.998 + 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()

ORIENTATION CONSTRAINT 넣어주기 

In [54]:


constraints = Constraints()

constraints.name = "constraints1"

oc1 = OrientationConstraint()

# input orientation constraints
constraints.orientation_constraints.append(oc1)
move_group.set_path_constraints(constraints)


pose_goal.position.x = 0.40
pose_goal.position.y = 0.44
pose_goal.position.z = 0.998 - 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()



In [124]:

import tf

tmp = tf.transformations.euler_from_quaternion([
    pose_goal.orientation.x,
    pose_goal.orientation.y,
    pose_goal.orientation.z,
    pose_goal.orientation.w])

[each*180/math.pi for each in tmp]

[100.36489387091633, -0.9390829294376395, -0.2363406321458759]

In [125]:
tf.transformations.quaternion_from_euler(30*math.pi/180,
                                         60*math.pi/180,
                                         90*math.pi/180)

array([-0.1830127,  0.5      ,  0.5      ,  0.6830127])

In [117]:
[p,q,r] = [each*math.pi/180 for each in [0., 60, 90.]]

x,y,z,w =tf.transformations.quaternion_from_euler(p,q,r)

NameError: name 'tf' is not defined

In [22]:
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()